## UCSB Thomas–Fermi notebook

- Uses `solvers/solverUCSB.py` and `mainUCSB.py`
- Edit the parameters below and run the cells to execute a single simulation
- Results are saved under `analysis_folder/<today>/<timestamp>_UCSB/case_00`


In [1]:
from pathlib import Path
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

# Import reusable pieces
from mainUCSB import build_vt_grid, magnetic_length_m
from solvers.solverUCSB import SimulationConfig, ThomasFermiSolver

import os

In [ ]:
# Parameters (match defaults in mainUCSB.py)

# Cluster flag: write to analysis_folder_Cluster if True
CLUSTER = False

# Grid
GRID_N = 32

# Magnetic field and domain
B_FIELD_T = 13.0  # Tesla
L_B_M = magnetic_length_m(B_FIELD_T)
HALF_SPAN_M = 20.0 * L_B_M
X_MIN_NM = -HALF_SPAN_M * 1e9
X_MAX_NM = +HALF_SPAN_M * 1e9
Y_MIN_NM = -HALF_SPAN_M * 1e9
Y_MAX_NM = +HALF_SPAN_M * 1e9

# X-mask
BAR_WIDTH_NM = 70.0

# Gate voltages
V_NS_EW_B_SETS = [(-0.10,0.60 ,0.10),(0,0.70,0)]  # Add more pairs as needed

# hBN thicknesses [nm]
D_T = 30.0
D_B = 30.0

# Optimisation
BASINHOPPING_NITER = 1
BASINHOPPING_STEP_SIZE = 0.1
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2_000_000

# Potential scaling/offset (typically keep as-is)
POTENTIAL_SCALE = 1.0
POTENTIAL_OFFSET = 0.0

# Exchange-correlation scaling
XC_SCALE = 1.8

# Progressive refinement (Matryoshka)
MATRYOSHKA = False
MATRYOSHKA_MIN_N = 32
COARSE_ACCEPT_LIMIT = 1

# Optional multi-case configuration (match mainUCSB)
PARALLEL = False  # Not recommended inside Jupyter
MAX_WORKERS = max(1, (os.cpu_count() or 2) - 1)

In [ ]:
# Run multiple cases sequentially using V_NS_EW_B_PAIRS (no parallel)
from dataclasses import asdict

base_dir = Path("analysis_folder_Cluster" if CLUSTER else "analysis_folder")
today = datetime.now().strftime("%Y%m%d")
date_dir = base_dir / today
date_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
batch_dir = date_dir / (timestamp + "_UCSB")
batch_dir.mkdir(parents=True, exist_ok=True)

for i, (vns, vew, vb) in enumerate(V_NS_EW_B_SETS):
    case_dir = batch_dir / f"case_{i:02d}"
    case_dir.mkdir(parents=True, exist_ok=True)

    # Build Vt for this pair
    vt_i = build_vt_grid(
        GRID_N,
        X_MIN_NM, X_MAX_NM,
        Y_MIN_NM, Y_MAX_NM,
        BAR_WIDTH_NM,
        vns, vew,
    )

    # Configure solver for this case
    cfg_i = SimulationConfig(
        Nx=GRID_N,
        Ny=GRID_N,
        B=B_FIELD_T,
        V_B=vb,
        dt=D_T*1e-9,
        db=D_B*1e-9,
        Vt_grid=vt_i,
        x_min_nm=X_MIN_NM,
        x_max_nm=X_MAX_NM,
        y_min_nm=Y_MIN_NM,
        y_max_nm=Y_MAX_NM,
        niter=BASINHOPPING_NITER,
        step_size=BASINHOPPING_STEP_SIZE,
        lbfgs_maxiter=LBFGS_MAXITER,
        lbfgs_maxfun=LBFGS_MAXFUN,
        potential_scale=POTENTIAL_SCALE,
        potential_offset=POTENTIAL_OFFSET,
        exc_file="data/0-data/Exc_data_new.csv",
        solver_type="solverUCSB",
        exc_scale=XC_SCALE,
        use_matryoshka=MATRYOSHKA,
        matryoshka_min_N=MATRYOSHKA_MIN_N,
        coarse_accept_limit=COARSE_ACCEPT_LIMIT,
    )

    # Save parameters pre-run
    with (case_dir / "simulation_parameters.txt").open("w", encoding="utf-8") as f:
        for k, v in asdict(cfg_i).items():
            f.write(f"{k} = {v}\n")

    # Optimise and save
    solver_i = ThomasFermiSolver(cfg_i)
    _ = solver_i.optimise()
    solver_i.plot_results(save_dir=str(case_dir), title_extra=f"V_NS={vns:+.2f} V, V_EW={vew:+.2f} V", show=False)
    solver_i.save_results(case_dir)

    print(f"Finished case {i:02d}: V_NS={vns:+.2f}, V_EW={vew:+.2f} → {case_dir}")

print(f"All sequential runs complete. Results stored in {batch_dir}")

basinhopping step 0: f -5962.94
basinhopping step 1: f -5963.38 trial_f -5963.38 accepted True lowest_f -5963.38
found new global minimum on step 1 with function value -5963.38
Finished case 00: V_NS=-0.10, V_EW=+0.60 → analysis_folder/20250813/20250813_181955_UCSB/case_00
basinhopping step 0: f -4630.72
basinhopping step 1: f -4631.45 trial_f -4631.45 accepted True lowest_f -4631.45
found new global minimum on step 1 with function value -4631.45
Finished case 01: V_NS=+0.00, V_EW=+0.70 → analysis_folder/20250813/20250813_181955_UCSB/case_01
All sequential runs complete. Results stored in analysis_folder/20250813/20250813_181955_UCSB


In [7]:
# Optional: run multiple cases (sequential in notebook)
# Uncomment to use
# from concurrent.futures import ProcessPoolExecutor, as_completed
# futures = []
# for i, (vns, vew) in enumerate(V_NS_EW_PAIRS):
#     pot_dir = batch_dir / f"case_{i:02d}"
#     pot_dir.mkdir(parents=True, exist_ok=True)
#     vt_i = build_vt_grid(
#         GRID_N,
#         X_MIN_NM, X_MAX_NM,
#         Y_MIN_NM, Y_MAX_NM,
#         BAR_WIDTH_NM,
#         vns, vew,
#     )
#     cfg_i = SimulationConfig(
#         Nx=GRID_N, Ny=GRID_N,
#         B=B_FIELD_T, V_B=V_B,
#         dt=D_T*1e-9, db=D_B*1e-9,
#         Vt_grid=vt_i,
#         x_min_nm=X_MIN_NM, x_max_nm=X_MAX_NM,
#         y_min_nm=Y_MIN_NM, y_max_nm=Y_MAX_NM,
#         niter=BASINHOPPING_NITER, step_size=BASINHOPPING_STEP_SIZE,
#         lbfgs_maxiter=LBFGS_MAXITER, lbfgs_maxfun=LBFGS_MAXFUN,
#         potential_scale=POTENTIAL_SCALE, potential_offset=POTENTIAL_OFFSET,
#         exc_file="data/0-data/Exc_data_new.csv",
#         solver_type="solverUCSB",
#         exc_scale=XC_SCALE,
#         use_matryoshka=MATRYOSHKA,
#         matryoshka_min_N=MATRYOSHKA_MIN_N,
#         coarse_accept_limit=COARSE_ACCEPT_LIMIT,
#     )
#     solver_i = ThomasFermiSolver(cfg_i)
#     _ = solver_i.optimise()
#     solver_i.plot_results(save_dir=str(pot_dir), title_extra=f"V_NS={vns:+.2f} V, V_EW={vew:+.2f} V", show=False)
#     solver_i.save_results(pot_dir)
# print(f"Saved batch outputs to: {batch_dir}")
